In [1]:
#importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt #visualization
%matplotlib inline 
#sns.set(color_codes=True)

In [2]:
#importing train datasets
df_train= pd.read_csv('C:/Users/tamak/Desktop/Data Science/Predicts/Predict 4/EDSA_ZM-Team1/train_set.csv')
#importing test datasets
df_test = pd.read_csv('C:/Users/tamak/Desktop/Data Science/Predicts/Predict 4/EDSA_ZM-Team1/test_set.csv')

Some research was performed to determine which months Golden Apples were harvested in South Africa. More information may be found at this link:
https://www.dennegeur.co.za/products/#:~:text=Golden%20Delicious%20was%20found%20as,to%20a%20golden%20yellow%20colour.


What did seem of interest was the following:

"Golden Delicious
Golden Delicious was found as a seedling in West Virginia, USA, in the 1880’s and introduced into South Africa in 1930 by Molteno Brothers of Grabouw. **Golden delicious is harvested from late February to mid March.** It is a medium to large apple, green when harvested, ripening to a golden yellow colour."

Here, a copy of the training set is created (as to not overwrite the origional dataset in case of errors).
The following columns, which seemed to have little correlation with avg_price_per_kg, were dropped:

- Commodities (after we subset the data to select only rows relevant to Golden Apples, this row becomes redundant)
- Province (the dummy variables columns created from this column were numerous and did not seem to greatly impact model performance)
- Size_Grade (the same reasoning for dropping 'Province')
- Container (the same reasoning for dropping 'Province' and 'Size_Grade')
- Date (after extracting the month for creation of 'Month' and 'Harvest_Seasons' columns)

As stated above, the 'Month' column is extracted from the date column and a new column 'Harvest_Season' is created (and dummy encoded) based on the research found.

In [3]:
model_train = df_train.loc[df_train['Commodities'] == 'APPLE GOLDEN DELICIOUS'].copy()

model_train = model_train.drop('Commodities',axis=1)

#model_train = model_train.drop('Province',axis=1)
model_train =  pd.get_dummies(model_train, drop_first = True, columns=['Province'])
#model_train = model_train.drop('Size_Grade',axis=1)
model_train =  pd.get_dummies(model_train, drop_first = True, columns=['Size_Grade'])
#model_train = model_train.drop('Container',axis=1)
model_train =  pd.get_dummies(model_train, drop_first = True, columns=['Container'])

model_train['Month'] = pd.DatetimeIndex(model_train['Date']).month
model_train = model_train.drop('Date',axis=1)

model_train["Harvest_Season"] = model_train["Month"].astype(str)
model_train["Harvest_Season"].replace({"1": "No", "2": "Yes", "12" : "No", "3" : "Yes", "4" : "No", "5" : "No", "6" : "No", "7": "No", "8" : "No", "9" : "No", "10" : "No", "11" : "No"}, inplace=True)
model_train =  pd.get_dummies(model_train, drop_first = True, columns=['Harvest_Season'])

model_train.head()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,avg_price_per_kg,Province_EASTERN CAPE,Province_NATAL,...,Container_EF120,Container_EG140,Container_IA400,Container_JE090,Container_JG110,Container_M4183,Container_M6125,Container_M9125,Month,Harvest_Season_Yes
1,18.3,150.0,170.0,51710.0,332,6075.6,822,8.51,0,0,...,0,0,0,0,0,1,0,0,9,0
7,11.0,50.0,50.0,16000.0,320,3520.0,0,4.55,0,0,...,0,0,0,0,1,0,0,0,4,0
24,9.0,55.0,55.0,990.0,18,162.0,1506,6.11,0,0,...,0,0,0,1,0,0,0,0,4,0
40,18.3,80.0,120.0,32020.0,388,7100.4,443,4.51,0,0,...,0,0,0,0,0,1,0,0,5,0
69,400.0,1800.0,1800.0,1800.0,1,400.0,2,4.50,1,0,...,0,0,1,0,0,0,0,0,9,0


The changes to the training set are also applied to a copy of the test set data - for consistancy with regards to the number of features to make predictions on once we have found our optimal model.

The test dataset also has an 'Index' column which also needs to be dropped to ensure consistancy

In [4]:
model_test = df_test.loc[df_test['Commodities'] == 'APPLE GOLDEN DELICIOUS'].copy()

model_test = model_test.drop('Commodities',axis=1)

#model_test = model_test.drop('Province',axis=1)
model_test =  pd.get_dummies(model_test, drop_first = True, columns=['Province'])
#model_test = model_test.drop('Size_Grade',axis=1)
model_test =  pd.get_dummies(model_test, drop_first = True, columns=['Size_Grade'])
#model_test = model_test.drop('Container',axis=1)
model_test =  pd.get_dummies(model_test, drop_first = True, columns=['Container'])

model_test['Month'] = pd.DatetimeIndex(model_test['Date']).month
model_test = model_test.drop('Date',axis=1)

model_test["Harvest_Season"] = model_test["Month"].astype(str)
model_test["Harvest_Season"].replace({"1": "No", "2": "Yes", "12" : "No", "3" : "Yes", "4" : "No", "5" : "No", "6" : "No", "7": "No", "8" : "No", "9" : "No", "10" : "No", "11" : "No"}, inplace=True)
model_test =  pd.get_dummies(model_test, drop_first = True, columns=['Harvest_Season'])
model_test = model_test.drop('Index',axis=1)

model_test.head()

,Weight_Kg,Low_Price,High_Price,Sales_Total,Total_Qty_Sold,Total_Kg_Sold,Stock_On_Hand,Province_EASTERN CAPE,Province_NATAL,Province_ORANGE FREE STATE,...,Container_EF120,Container_EG140,Container_IA400,Container_JE090,Container_JG110,Container_M4183,Container_M6125,Container_M9125,Month,Harvest_Season_Yes
0,12.0,128.0,136.0,5008.0,38,456.0,0,0,0,0,...,0,0,0,0,0,0,0,0,7,0
1,18.3,220.0,220.0,1760.0,8,146.4,2,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,12.0,120.0,120.0,720.0,6,72.0,45,0,0,0,...,0,0,0,0,0,0,0,0,8,0
3,18.3,160.0,160.0,160.0,1,18.3,8,0,0,0,...,0,0,0,0,0,1,0,0,5,0
4,18.3,140.0,160.0,14140.0,100,1830.0,19,0,0,0,...,0,0,0,0,0,1,0,0,5,0


Here train_test_split is imported from Sci-kit Learn and data is divided into training and testing data.

*StandardScaler was also imported and fit but seemed to lower model accuracy so it was left out*

A few other points to consider:

- Sci-kit Learn metrics were also imported to calculate performance metrics.
- A test_size of 33% was decided upon as the 'missing' training data on Kaggle also comprised of 33% of the data 
- A random_state of 42 was chosen

In [5]:
from sklearn.model_selection import train_test_split as tts
from sklearn import metrics
#from sklearn.preprocessing import StandardScaler
import math

y = model_train.avg_price_per_kg #defining label
x = model_train.drop('avg_price_per_kg',axis=1) #defining features

# create scaler
#scaler = StandardScaler().fit(x)
# scaling features
#X_scaled = scaler.transform(x)

x_train, x_test, y_train, y_test = tts(x, y, test_size=0.05, random_state=42) # creating train-test-split

# converting series to arrays
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

Here LinearRegression is imported from Sci-kit Learn. The model is fitted and it's performance metrics are calculated:

In [6]:
from sklearn.linear_model import LinearRegression

model_lr = LinearRegression() 
model_lr.fit(x_train, y_train) 

print("Train MSE " ,metrics.mean_squared_error(y_train, model_lr.predict(x_train)))
print("Test MSE " ,metrics.mean_squared_error(y_test, model_lr.predict(x_test)))
print("RMSE ", math.sqrt(metrics.mean_squared_error(y_test, model_lr.predict(x_test))))

Train MSE  1.891157457697162
Test MSE  1.776342385583127
RMSE  1.332794952565145


Based on these metrics, the Linear Regression Model does not appear to be performing well...

Here Lasso is imported from Sci-kit Learn. The model is fitted and it's performance metrics are calculated:

In [7]:
from sklearn.linear_model import Lasso

model_las = Lasso(alpha=0.1, random_state=42)
model_las.fit(x_train, y_train)

print("Train MSE " ,metrics.mean_squared_error(y_train, model_las.predict(x_train)))
print("Test MSE " ,metrics.mean_squared_error(y_test, model_las.predict(x_test)))
print("RMSE ", math.sqrt(metrics.mean_squared_error(y_test, model_las.predict(x_test))))

Train MSE  2.616863906377461
Test MSE  2.7734931544345223
RMSE  1.6653807836151233


Based on these metrics, the Lasso Model appears to be performing even worse than the Linear Regression Model. The lowest scores for the Lasso Model were calcualted at 0.1 alpha. An alpha of 0 is essentially a Linear Model... 

Here Ridge is imported from Sci-kit Learn. The model is fitted and it's performance metrics are calculated:

In [8]:
from sklearn.linear_model import Ridge
model_ridge = Ridge(alpha=0.1, random_state=42)
model_ridge.fit(x_train, y_train)

print("Train MSE " ,metrics.mean_squared_error(y_train, model_ridge.predict(x_train)))
print("Test MSE " ,metrics.mean_squared_error(y_test, model_ridge.predict(x_test)))
print("RMSE ", math.sqrt(metrics.mean_squared_error(y_test, model_ridge.predict(x_test))))

Train MSE  1.891220090847978
Test MSE  1.7770616227451284
RMSE  1.3330647481443383


As with the Lasso Model, the Ridge Model seems to perform best with a low Alpha, indicating a stronger Linear Model potentially. Ridge does appear to be performing better than Lasso, if only slightly.

Here we import RandomForestRegressor from Sci-kit Learn and fit a model. The paramaters which performed the best (on average -through trial-and-error) seemed to be the settings of:

- max_features = 'auto'
- vebose = 1

In [9]:
from sklearn.ensemble import RandomForestRegressor 
from sklearn.metrics import mean_squared_error as MSE

model_rf = RandomForestRegressor(max_features="auto", verbose=1)
model_rf.fit(x_train,y_train)

print("Best Case Train MSE - Random Forest: " ,metrics.mean_squared_error(y_train, model_rf.predict(x_train)))
print("Best Case Test MSE - Random Forest: " ,metrics.mean_squared_error(y_test, model_rf.predict(x_test)))
print("Best Case RMSE - Random Forest: ", math.sqrt(metrics.mean_squared_error(y_test, model_rf.predict(x_test))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Best Case Train MSE - Random Forest:  0.05051964297195234
Best Case Test MSE - Random Forest:  0.2116736134693878
Best Case RMSE - Random Forest:  0.46008000768278096


Here one may observe that the Random Forest Model appears to be performing the best so far.

Here we import DecisionTreeRegressor from Sci-kit Learn and fit a model. The paramaters which performed the best (on average -through trial-and-error) seemed to be the settings of:

- max_features = 'log2'
- random_state = 42 (This is just to keep in line with a consistent random_state across all training sets)

In [10]:
from sklearn.tree import DecisionTreeRegressor

model_dt = DecisionTreeRegressor(random_state=42, max_features='log2')
model_dt.fit(x_train,y_train)

print("Best Case Train MSE - Decision Tree: " ,metrics.mean_squared_error(y_train, model_dt.predict(x_train)))
print("Best Case Test MSE - Decision Tree: " ,metrics.mean_squared_error(y_test, model_dt.predict(x_test)))
print("Best Case RMSE - Decision Tree: ", math.sqrt(metrics.mean_squared_error(y_test, model_dt.predict(x_test))))

Best Case Train MSE - Decision Tree:  3.489028814893364e-32
Best Case Test MSE - Decision Tree:  1.1273438775510203
Best Case RMSE - Decision Tree:  1.0617645113446863


As we can see the Decision Tree Model definately appears to perform worse than the Random Forest Model but better than the Ridge and Lasso Models.

Here we import BaggingRegressor for Sci-kit Learn.

We set the following parameters to try and obtain the lowest scores:

- n_estimators = 1000
- bootstrap = True
- random_state = 42
- warm_start = False

In [11]:
from sklearn.ensemble import BaggingRegressor

bag_reg = BaggingRegressor(n_estimators = 1000, warm_start = False, n_jobs = -1, bootstrap = True, random_state =42)
bag_reg.fit(x_train,y_train)

print("Best Case Train MSE - Bagging: " ,metrics.mean_squared_error(y_train, bag_reg.predict(x_train)))
print("Best Case Test MSE - Bagging: " ,metrics.mean_squared_error(y_test, bag_reg.predict(x_test)))
print("Best Case RMSE - Bagging: ", math.sqrt(metrics.mean_squared_error(y_test, bag_reg.predict(x_test))))

Best Case Train MSE - Bagging:  0.052354120639644446
Best Case Test MSE - Bagging:  0.21131087683367342
Best Case RMSE - Bagging:  0.4596856282653107


This model also appear to be performing better than most. The only other comparable model is the Random Forest at this point...

Here we import BaggingRegressor for Sci-kit Learn.

We set the following parameters to try and obtain the lowest scores:

- random_state = 42
- base_estimator = model_dt (Decision Tree, Random Forest may have proven better but took to much time to run) 
- n_estimators = 100
- learning_rate = 0.15

In [19]:
from sklearn.ensemble import AdaBoostRegressor

bst_reg = AdaBoostRegressor(random_state=42, base_estimator = model_dt, n_estimators=100)
bst_reg.fit(x_train,y_train)

print("Best Case Train MSE - Boost: " ,metrics.mean_squared_error(y_train, bst_reg.predict(x_train)))
print("Best Case Test MSE - Boost: " ,metrics.mean_squared_error(y_test, bst_reg.predict(x_test)))
print("Best Case RMSE - Boost: ", math.sqrt(metrics.mean_squared_error(y_test, bst_reg.predict(x_test))))

Best Case Train MSE - Boost:  0.0004216828478964407
Best Case Test MSE - Boost:  0.28421326530612245
Best Case RMSE - Boost:  0.5331165588369231


We can see that this model is also performing relatively well - other than on the actual training set.

Here we import VotingRegressor from Sci-kit Learn. We also select the models which have obtained the best accuaracy thus far:
    
- AdaBoost (bst_reg)
- Bagging (bag_reg)
- Random Forest (model_rf)

And calculate the metrics:

In [26]:
from sklearn.ensemble import VotingRegressor

# Define the models which we'll include in our ensemble. 

models = [("Boost",bst_reg),("Bagging",bag_reg),("Random Forest",model_rf)]

# Specify weights for weighted model averaging
model_weightings = np.array([0.3,0.4,0.3])

v_reg = VotingRegressor(estimators=models,weights=model_weightings)
v_reg.fit(x_train,y_train)

print("Best Case Train MSE - Voting: " ,metrics.mean_squared_error(y_train, v_reg.predict(x_train)))
print("Best Case Test MSE - Voting: " ,metrics.mean_squared_error(y_test, v_reg.predict(x_test)))
print("Best Case RMSE - Voting: ", math.sqrt(metrics.mean_squared_error(y_test, v_reg.predict(x_test))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Best Case Train MSE - Voting:  0.02528189757946286


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Best Case Test MSE - Voting:  0.2037024317072652
Best Case RMSE - Voting:  0.451334057774577


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Here we import StackingRegressor from Sci-kit Learn and try once more to obtain a better model

In [27]:
from sklearn.ensemble import StackingRegressor

meta_learner_reg = LinearRegression()

s_reg = StackingRegressor(estimators=models, final_estimator=meta_learner_reg, passthrough = True,cv=10, n_jobs=-1) 
s_reg.fit(x_train,y_train)

print("Best Case Train MSE - Stacking: " ,metrics.mean_squared_error(y_train, s_reg.predict(x_train)))
print("Best Case Test MSE - Stacking: " ,metrics.mean_squared_error(y_test, s_reg.predict(x_test)))
print("Best Case RMSE - Stacking: ", math.sqrt(metrics.mean_squared_error(y_test, s_reg.predict(x_test))))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Best Case Train MSE - Stacking:  0.03889638601709057


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


Best Case Test MSE - Stacking:  0.18887027624492467
Best Case RMSE - Stacking:  0.43459208028325214


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


It appears as though the Stacking Model performs best over all and we use this model to predict the test set of the data:

In [28]:
y_pred = s_reg.predict(model_test)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished


The variable 'Index' is set to the index of df_test to ensure correct submission of rows in test dataset.

In [29]:
index = df_test['Index']

The 'submission' variable is used to create a dataframe to be exported:

In [30]:
submission = pd.DataFrame({'Index' : index, 'avg_price_per_kg' : y_pred})

The dataframe is then exported to a csv file for uploading to Kaggle:

In [31]:
submission.to_csv(r'C:\Users\tamak\Desktop\Data Science\Predicts\Predict 4\EDSA_ZM-Team1\KaggleCSV.csv', index = False)